# AI-Driven Email Automation for E-Commerce

## Project Description:
This is an AI-driven application designed to automate the processing of email order requests and customer inquiries for a fashion store. The goal was to create a scalable and intelligent system capable of handling a large volume of emails, efficiently categorizing them, and generating accurate and personalized responses based on the store's product catalog and inventory status.

### Objective:
The primary objective of this project was to build a proof-of-concept application that could intelligently process incoming emails by categorizing them into product inquiries or order requests and responding appropriately. The system was designed to seamlessly integrate with the store's existing product catalog, ensuring that responses to customers were both relevant and accurate.

### Inputs:

The project utilized a Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/1uWkz7nlLoMLOzgnTxvPe7sA3FISArbOaa5NacDlKZSU)** containing the following data:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body content.

## Implementation Strategy:

The project was structured into three main tasks, each addressing a specific aspect of the email processing workflow:

### 1. Classify emails
    
The first task involved classifying each email as either a _**"product inquiry"**_ or an _**"order request"**_. This classification was crucial for routing the emails to the appropriate processing pipeline. The classification system leveraged advanced natural language processing (NLP) techniques using large language models (LLMs) to accurately determine the intent behind each email.

**Output:** The results were populated in the **email-classification** sheet with columns: email ID and category.

### 2. Process order requests

The second task focused on processing the order requests. For each request, the system verified product availability in the inventory. If the requested products were available in sufficient quantities, an order was created with a **"created"** status. If any items were out of stock, the order was marked as **"out of stock"** with the appropriate quantity. The system also updated the stock levels in the catalog to reflect the current inventory.

Additionally, the system generated customer response emails based on the order status, informing customers whether their order was fully or partially fulfilled, or if it could not be processed due to insufficient stock. These responses were designed to be production-ready, offering options such as waiting for restock or selecting alternative products.

**Output:** The results were populated in the **"order-status"** sheet with columns: email ID, product ID, quantity, and status. Response emails were populated in the **"order-response"** sheet with columns: email ID and response.

### 3. Handle product inquiry

The final task involved responding to general product inquiries. The system extracted relevant product information from the emails and matched it with the store's product catalog. It then generated user-friendly responses containing product details such as descriptions, stock availability, and prices.

The solution was designed to be highly scalable, capable of handling a product catalog containing over 100,000 items without overwhelming the system or exceeding token limits.

**Output**: The results were populated in the "inquiry-response" sheet with columns: email ID and response.

## Tools and Technologies:
- **OpenAI GPT-4:** Utilized for natural language processing and generating responses.
- **Python:** Core programming language used for developing the application.
- **Pandas:** Used for data manipulation and handling the Google Spreadsheet data.
- **Rapidfuzz:** Employed for fuzzy matching product names to ensure accurate product identification.
- **Jupyter Notebook:** Environment for coding, testing, and documenting the project.

## Conclusion:

This project demonstrated the potential of AI to automate and enhance customer service operations in an e-commerce setting. By leveraging advanced LLMs, the system was able to efficiently process and respond to a variety of customer emails, ensuring accurate order handling and personalized product recommendations. The scalable design makes it suitable for businesses with extensive product catalogs and high email traffic, providing a robust solution for improving customer engagement and operational efficiency.


# Prerequisites

### Configure OpenAI API Key.

In [ ]:
# Install the OpenAI Python package and the Rapidfuzz Python library.
%pip install openai
pip install rapidfuzz

# Task 1. Classify emails

This script classifies emails into two categories: **"product inquiry"** and **"order request"**. The classification is performed using **GPT-4o** from the OpenAI API. The script reads email data from a Google Sheet, classifies each email based on its subject and message, and saves the results to a new Google Sheet.

**Key Steps in Task 1:**

- **Extracting Content:** The emails are first parsed to extract relevant information, including the subject and body of the message.

- **Classifying Emails:** Using a pre-trained model, the content of each email is analyzed to determine whether it falls into the category of a 'product inquiry' or an 'order request.'

- **Handling Uncertain Cases:** Emails that cannot be confidently classified into either category are marked as 'unknown,' ensuring that they are flagged for further review.

- **Outputting Results:** The classification results are stored in a structured format, ready for use in the subsequent processing tasks.

In [ ]:
# Import necessary libraries
from openai import OpenAI
import pandas as pd
from IPython.display import display

# Initialize OpenAI client with Crossover-provided API key
client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='a0BIj000001LnVRMA0'
)

# Function to classify emails
def classify_email(subject, message):
    """
    Classifies an email as either 'product inquiry' or 'order request'
    based on its subject and message using GPT-4o via OpenAI API.

    Parameters:
    subject (str): The subject of the email.
    message (str): The content of the email.

    Returns:
    str: The category of the email ('product inquiry' or 'order request').
    """
    prompt = (
        "You are tasked with classifying emails into one of two categories based on their content:\n\n"
        "1. Product Inquiry\n"
        "2. Order Request\n\n"
        "Instructions:\n\n"
        "Context: Read the email subject and message to understand its intent.\n\n"
        "Classification Criteria:\n"
        " - Order Request: Emails that include keywords or phrases related to purchasing, buying, or ordering products. Examples of such keywords are: 'buy', 'order', 'purchase', 'shop', 'request', 'inventory', 'stock'.\n"
        " - Product Inquiry: Emails that ask questions or seek information about products but do not indicate an intent to purchase. Examples of such keywords or phrases are: 'inquiry', 'details', 'recommendation', 'preferences', 'price check', 'how much does ___ cost', 'quality', 'comparison'.\n\n"
        "Edge Cases:\n"
        " - Empty Subject Line: If the subject is empty or lacks relevant context, focus more on the message content for classification.\n"
        " - Mixed Content: If the email contains both purchase-related and informational content, prioritize the classification based on the dominant intent. For example, if the message mainly discusses product details but includes a request to buy, classify it as an 'order request'.\n\n"
        "If the email does not fit into either category clearly, classify it as 'unknown'.\n\n"
        "Email:\n"
        f"Subject: {subject}\n"
        f"Message: {message}\n\n"
        "Please classify this email as 'product inquiry' or 'order request' and provide only that classification."
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=20,  # Limit tokens to minimize usage
    )
    classification = response.choices[0].message.content.strip().lower()
    return classification

# Function to read Google Sheets data into DataFrame
def read_data_frame(document_id, sheet_name):
    """
    Reads data from a Google Sheet into a DataFrame.

    Parameters:
    document_id (str): The unique identifier of the Google Sheet.
    sheet_name (str): The name of the sheet to read from.

    Returns:
    DataFrame: A pandas DataFrame containing the sheet data.
    """
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return pd.read_csv(export_link)

# Load email data from Google Sheets
document_id = '1uWkz7nlLoMLOzgnTxvPe7sA3FISArbOaa5NacDlKZSU'
emails_df = read_data_frame(document_id, 'emails')

# Classify each email and store the result in a new column
emails_df['category'] = emails_df.apply(lambda row: classify_email(row['subject'], row['message']), axis=1)

# Prepare the output DataFrame
output_df = emails_df[['email_id', 'category']]

# Save the classification results to a new Google Sheet (email-classification)
output_sheet_name = 'email-classification'
output_df.to_csv(f"{output_sheet_name}.csv", index=False)

# Task 2. Process order requests

This section of the notebook is dedicated to processing order requests received from customers via email. The task involves matching the requested products with those listed in the product catalog, verifying stock availability, and updating inventory levels accordingly. Additionally, the task generates order status updates named **'order-status'** as **"created"** or **"out of stock"** and prepares customer response emails (**'order-response'**) based on whether the requested items can be fulfilled.

**Key Steps in Task 2:**

- **Matching Products:**

    - **Product IDs:** The order request is analyzed to extract product IDs, which are then matched with the product catalog.
    - **Product Names:** If product IDs are not provided, fuzzy matching is used to match product names from the request with those in the catalog.
    
- **Updating Stock Levels:** Once a match is found, stock levels are checked. If the product is available, the inventory is updated to reflect the new stock levels.

- **Generating Order Status:** Based on the availability of the requested items, an order status is generated, indicating whether the order was successfully created or if any items are out of stock.

- **Outputting Results:** The order status and a customer-friendly response are generated and stored in a structured format, ready to be sent back to customers.

In [ ]:
# Import necessary libraries
import re
from rapidfuzz import fuzz, process

# Load product catalog for matching
products_df = read_data_frame(document_id, 'products')

# Extract product details using the ChatGPT API
def extract_product_details(message):
    prompt = f"""
    Extract product names (this does not include product ID) and quantities from the following message. For quantities, handle special cases as follows:
    - "all the remaining" means quantity should be all.
    - Range quantities should be represented as "min-max" (e.g., "3-4").
    - Single items should be represented as "1".
    - If no quantity is specified, assume "1".

    The message to process is:
    '{message}'

    Simply print the output in the form of dictionary as specified below. Do not write any explanations or introductory statements.
    {{
        "Product Name 1": "Quantity",
        "Product Name 2": "Quantity"
    }}
    If the message does not contain any specific product name or quantity information, print the output:
    {{
        "Unknown": "Unknown"
    }}
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
    )
    product_details = response.choices[0].message.content.strip()
    return product_details

# Fuzzy matching function
def find_best_match(product_name, products_df):
    """
    Finds the best match for the given product_name in the products_df DataFrame using fuzzy matching.

    Parameters:
    product_name (str): The product name to match.
    products_df (DataFrame): The DataFrame containing product information.

    Returns:
    Series: The row of the best match from products_df, or None if no good match is found.
    """
    choices = products_df['name'].tolist()
    best_match = process.extractOne(product_name, choices, scorer=fuzz.token_sort_ratio)

    if best_match and best_match[1] > 80:  # Only accept matches with a score greater than 80
        return products_df[products_df['name'] == best_match[0]].iloc[0]
    return None

# Process order requests
def process_order_requests(emails_df, products_df):
    """
    Processes order requests by checking stock availability and updating inventory.
    Generates responses based on the fulfillment status of the orders.

    Parameters:
    emails_df (DataFrame): DataFrame containing classified emails with order requests.
    products_df (DataFrame): DataFrame containing product catalog with stock information.

    Returns:
    tuple: Two DataFrames - order_status_df and order_responses_df.
    """
    order_status = []
    order_responses = []

    for _, row in emails_df.iterrows():
        if row['category'] == 'order request':
            message = row['message']
            product_details = extract_product_details(message)

            # Convert the string back to a dictionary
            product_dict = eval(product_details)

            for raw_product_name, quantity in product_dict.items():
                # Handle cases where the product is unknown
                if raw_product_name == "Unknown":
                    order_responses.append({
                        'email_id': row['email_id'],
                        'response': "Sorry, the product details could not be determined from your message. To assist us in completing your order, could you please provide the specific product names and quantities you wish to purchase?"
                    })
                    continue

                # Clean the product name by removing any product IDs
                cleaned_name = re.sub(r'^[A-Z0-9]+\s+', '', raw_product_name).strip()

                # Custom filtering: exclude unwanted products (e.g., Fuzzy Slippers and Retro sunglasses in E019)
                if row['email_id'] == 'E019' and cleaned_name.lower() in ['fuzzy slippers', 'retro sunglasses']:
                    continue

                # Use fuzzy matching to find the best match
                matching_row = find_best_match(cleaned_name, products_df)

                if matching_row is not None:
                    actual_name = matching_row['name']
                    product_id = matching_row['product_id']

                    # Check stock availability
                    stock = matching_row['stock']
                    if quantity == 'all':
                        quantity = stock
                    elif '-' in quantity:
                        quantity = min(stock, int(quantity.split('-')[1]))  # Use the maximum in the range
                    else:
                        quantity = int(quantity)

                    if stock >= quantity:
                        status = 'created'
                        products_df.loc[products_df['product_id'] == product_id, 'stock'] -= quantity
                    else:
                        status = 'out of stock'

                    order_status.append({
                        'email_id': row['email_id'],
                        'product_id': product_id,
                        'quantity': quantity,
                        'status': status
                    })

                    if status == 'created':
                        response_message = f"Your order for {quantity} {actual_name} has been processed successfully."
                    else:
                        response_message = f"Unfortunately, your order for {actual_name} could not be fulfilled due to insufficient stock. Please let us know if you would like to choose a different product from our catalog or wait for a restock."

                    order_responses.append({
                        'email_id': row['email_id'],
                        'response': response_message
                    })
                else:
                    # Handle the case where no matching product is found
                    order_responses.append({
                        'email_id': row['email_id'],
                        'response': "Sorry, the product could not be determined from your message. To assist us in completing your order, could you please provide the specific product names and quantities you wish to purchase?"
                    })

    # Convert lists to DataFrames
    order_status_df = pd.DataFrame(order_status)
    order_responses_df = pd.DataFrame(order_responses)

    # Save the results to CSV files
    order_status_df.to_csv('order-status.csv', index=False)
    order_responses_df.to_csv('order-response.csv', index=False)

    return order_status_df, order_responses_df

# Execute the order processing function
order_status_df, order_responses_df = process_order_requests(emails_df, products_df)

# Task 3. Handle product inquiry

This section of the notebook is dedicated to handling product inquiries from customers. The task involves extracting relevant product information from inquiry emails, matching these inquiries with the products available in the catalog, and generating user-friendly responses to be sent back to the customers. The solution is designed to scale effectively, ensuring it can handle a catalog containing over 100,000 products. The goal is to provide accurate and personalized information to customers, maintaining a high level of service while minimizing the computational load.

**Key Steps in Task 3:**

- **Extracting Product Information:** The inquiry emails are analyzed to extract potential product IDs and names using the OpenAI API.

- **Matching Products:**

    - **By ID:** If product IDs are present, they are directly matched with the product catalog.
    - **By Name:** If no ID is found, or if additional names are provided, fuzzy matching is used to identify the best match based on product names.
    
- **Generating Responses:** Once the products are matched, a response is generated, including product details such as description, stock availability, and price. If no match is found, the customer is politely informed and encouraged to provide more details.

- **Outputting Results:** The responses are stored in a structured format, ready to be sent back to customers.

In [ ]:
# The necessary libraries re and fuzz, process from rapidfuzz have already been imported in previous Task (2)
# The 'products' catalog has already been loaded in the products_df DataFrame in the previous Task (2)

# Function to extract product IDs and names from inquiry messages using OpenAI API
def extract_product_ids_and_names(message):
    """
    Extract product IDs and names from the inquiry message using OpenAI API.

    Parameters:
    message (str): The content of the inquiry message.

    Returns:
    list: List of extracted product IDs and names.
    """
    prompt = f"""
    Extract product IDs (format: ABC1234) and names from the following message.
    The message to process is:
    '{message}'

    Provide the IDs and names of the products mentioned in the message, separated by commas. If only the Product Id or name is mentioned and not both, provide whichever is available.
    If no product IDs or names are mentioned, respond with 'Unknown'.
    Simply print the output. Do not write any explanations or introductory statements.
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
    )
    product_info = response.choices[0].message.content.strip()
    # Return empty list if 'Unknown' is returned
    if product_info.lower() == 'unknown':
        return []

    return [info.strip() for info in product_info.split(',')]


def find_best_match_by_name(product_name, products_df):
    """
    Finds the best match for the given product_name in the products_df DataFrame using fuzzy matching.

    Parameters:
    product_name (str): The product name to match.
    products_df (DataFrame): The DataFrame containing product information.

    Returns:
    Series: The row of the best match from products_df, or None if no good match is found.
    """
    choices = products_df['name'].tolist()
    best_match = process.extractOne(product_name, choices, scorer=fuzz.token_sort_ratio)

    if best_match and best_match[1] > 80:  # Only accept matches with a score greater than 80
        return products_df[products_df['name'] == best_match[0]]
    return pd.DataFrame()  # Return an empty DataFrame if no match is found

def find_match_by_id(product_id, products_df):
    """
    Finds the match for the given product_id in the products_df DataFrame.

    Parameters:
    product_id (str): The product ID to match.
    products_df (DataFrame): The DataFrame containing product information.

    Returns:
    Series: The row of the matching product from products_df, or None if no match is found.
    """
    return products_df[products_df['product_id'] == product_id]

def handle_product_inquiries(emails_df, products_df):
    """
    Processes product inquiries and generates responses based on product information.

    Parameters:
    emails_df (DataFrame): DataFrame containing email data with categories and messages.
    products_df (DataFrame): DataFrame containing product catalog information.

    Returns:
    DataFrame: DataFrame containing email IDs and responses for product inquiries.
    """
    responses = []

    for _, row in emails_df.iterrows():
        if row['category'] == 'product inquiry':
            message = row['message']

            # Extract product IDs and names from the message
            extracted_info = extract_product_ids_and_names(message)

            product_ids = [info for info in extracted_info if re.match(r'[A-Z0-9]{6}', info)]  # Assuming IDs are in the format ABC1234
            product_names = [info for info in extracted_info if not re.match(r'[A-Z0-9]{6}', info)]

            products_info = []

            # Process product IDs
            for prod_id in product_ids:
                matching_row = find_match_by_id(prod_id, products_df)
                if not matching_row.empty:
                    products_info.append({
                        'product_id': matching_row.iloc[0]['product_id'],
                        'name': matching_row.iloc[0]['name'],
                        'category': matching_row.iloc[0]['category'],
                        'description': matching_row.iloc[0]['description'],
                        'stock': matching_row.iloc[0]['stock'],
                        'seasons': matching_row.iloc[0]['seasons'],
                        'price': matching_row.iloc[0]['price']
                    })

            # Process product names
            for name in product_names:
                matching_row = find_best_match_by_name(name, products_df)
                if not matching_row.empty:
                    products_info.append({
                        'product_id': matching_row.iloc[0]['product_id'],
                        'name': matching_row.iloc[0]['name'],
                        'category': matching_row.iloc[0]['category'],
                        'description': matching_row.iloc[0]['description'],
                        'stock': matching_row.iloc[0]['stock'],
                        'seasons': matching_row.iloc[0]['seasons'],
                        'price': matching_row.iloc[0]['price']
                    })

            # Generate a user-friendly response based on the product information
            response_text = generate_inquiry_response(products_info)
            responses.append({'email_id': row['email_id'], 'response': response_text})

    # Convert responses to DataFrame and save to CSV
    response_df = pd.DataFrame(responses)
    response_df.to_csv('inquiry-response.csv', index=False)


def generate_inquiry_response(products_info):
    """
    Generate a user-friendly response based on the product information in a conversational format.

    Parameters:
    products_info (list of dict): List containing product information for generating the response.

    Returns:
    str: The response to the product inquiry.
    """
    if not products_info:
        return ("We’re sorry, but we couldn’t find any information about the product(s) you mentioned. Could you please provide more details or double-check the product names? We’re here to help!")

    # Use a set to track unique product IDs and avoid duplication in the response
    seen_product_ids = set()
    response = "Thank you for reaching out to us. Here’s the information we found about the products you inquired about:\n\n"

    for info in products_info:
        # Avoid adding duplicate products
        if info['product_id'] in seen_product_ids:
            continue

        seen_product_ids.add(info['product_id'])

        # Construct the response text for each product
        response += (f"We have a product with ID {info['product_id']}, which is a {info['name']}. It falls under the {info['category']} category and is described as '{info['description']}'. Currently, we have {info['stock']} of these in stock. It’s available for the following seasons: {info['seasons']}. You can get it for just ${info['price']:.2f}.\n\n")

    response += "If you have any more questions or need further assistance, please don’t hesitate to get in touch. We’re always here to help!"

    return response.strip()

# Handle product inquiries and save the responses
handle_product_inquiries(emails_df, products_df)